# 03. Chạy Thí nghiệm & Thu thập Kết quả

**Mục tiêu:** Đây là notebook "chính" của dự án. Nó sẽ:

1.  Định nghĩa một danh sách các vấn đề TSP (ví dụ: `berlin52`, `burma14`, `eil76`).
2.  Gọi **tất cả 10 thuật toán** từ thư mục `/algorithms/`.
3.  Chạy từng thuật toán trên từng vấn đề.
4.  Đo lường **thời gian chạy** (`time`) và **chi phí lộ trình** (`cost`).
5.  Tính toán **khoảng cách %** (`gap %`) so với giải pháp tối ưu (nếu có).
6.  Lưu tất cả dữ liệu thô vào một file `results/results.csv` duy nhất bằng `pandas`.

Notebook này được thiết kế để chạy trong thời gian dài (có thể mất vài phút đến vài giờ tùy thuộc vào cấu hình).

**LƯU Ý QUAN TRỌNG:** Các thuật toán Exact (`Brute Force`, `Held-Karp`) sẽ được **bỏ qua (skipped)** một cách có chủ đích trên các vấn đề lớn (`N > 10` hoặc `N > 21`) để tránh thời gian chạy vô tận.

## 1. Thiết lập & Imports

Chúng ta sẽ import `pandas` (để xử lý data) và `time` (để đo thời gian) cùng với tất cả các module thuật toán và tiện ích của chúng ta.

In [ ]:
import sys
import os
import time
import pandas as pd
import numpy as np

# --- Cấu hình sys.path --- 
# Thêm thư mục gốc của dự án (cao hơn 'notebooks' một cấp) vào sys.path
# để chúng ta có thể import các module 'utils' và 'algorithms'
project_root = os.path.normpath(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Đã thêm '{project_root}' vào sys.path")
else:
    print(f"'{project_root}' đã có trong sys.path")
    
# --- Import Utils --- 
from utils import data_loader
from utils import evaluator

# --- Import 10 Thuật toán --- 
# Exact
from algorithms import brute_force
from algorithms import held_karp
# Heuristics Cổ điển
from algorithms import nearest_neighbor
from algorithms import nearest_insertion
# Xấp xỉ
from algorithms import christofides
# Cải tiến
from algorithms import two_opt
# Metaheuristics
from algorithms import simulated_annealing
from algorithms import genetic_algorithm
from algorithms import ant_colony
from algorithms import tabu_search

print("Tất cả các module đã được import thành công.")

## 2. Cấu hình Thí nghiệm

Định nghĩa các vấn đề chúng ta muốn chạy và các siêu tham số (hyperparameters) cho các thuật toán Metaheuristic.

In [ ]:
# --- DANH SÁCH VẤN ĐỀ ĐỂ CHẠY --- 
# Thêm bất kỳ file .tsp nào bạn có trong /data/tsplib/
# 'test_10' (N=10) - để chạy Brute Force
# 'burma14' (N=14) - để chạy Held-Karp
# 'berlin52' (N=52) - vấn đề chuẩn
# 'eil76' (N=76) - vấn đề lớn hơn
PROBLEMS_TO_RUN = ['test_10', 'burma14', 'berlin52', 'eil76']

# --- ĐƯỜNG DẪN --- 
DATA_DIR = '../data'
RESULTS_FILE = '../results/results.csv'

# --- SIÊU THAM SỐ (HYPERPARAMETERS) --- 
# Đây là các giá trị "khởi điểm". Chúng ta sẽ phân tích chúng sau.

SA_PARAMS = {
    'initial_temp': 10000.0,
    'cooling_rate': 0.999,
    'min_temp': 0.1,
    'max_iterations_per_temp': 100
}

GA_PARAMS = {
    'population_size': 100,
    'num_generations': 500,
    'mutation_rate': 0.02,
    'elite_size': 10,
    'tournament_k': 5
}

ACO_PARAMS = {
    'num_ants': 50, # Số lượng kiến (bằng N)
    'num_iterations': 100,
    'alpha': 1.0, 
    'beta': 5.0, # Ưu tiên khoảng cách ngắn
    'evaporation_rate': 0.5,
    'pheromone_deposit': 100.0
}

TS_PARAMS = {
    'max_iterations': 2000,
    'tabu_tenure': 30 # Bằng khoảng N/2
}

## 3. Vòng lặp Thí nghiệm Chính

Đây là nơi công việc nặng nhọc diễn ra. Chúng ta lặp qua từng vấn đề, sau đó lặp qua từng thuật toán, ghi lại kết quả.

In [ ]:
all_results = []

print("--- BẮT ĐẦU CHẠY THÍ NGHIỆM ---\n")

for problem_name in PROBLEMS_TO_RUN:
    print(f"\n{'='*10} ĐANG XỬ LÝ: {problem_name.upper()} {'='*10}")
    
    # 1. Tải dữ liệu
    try:
        problem_data = data_loader.load_problem(problem_name, DATA_DIR)
        matrix = problem_data['matrix']
        dim = problem_data['dimension']
        
        opt_cost = 0
        if problem_data['optimum_tour']:
            opt_cost = evaluator.calculate_tour_cost(problem_data['optimum_tour'], matrix)
        print(f"Đã tải {problem_name} (N={dim}), Chi phí tối ưu: {opt_cost}")
        
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file cho {problem_name}. Bỏ qua.")
        continue

    # Hàm trợ giúp để ghi kết quả
    def record_result(algo_name, tour, cost, exec_time):
        gap = ((cost - opt_cost) / opt_cost) * 100 if opt_cost > 0 else 0
        all_results.append({
            'problem': problem_name,
            'dimension': dim,
            'algorithm': algo_name,
            'cost': int(cost),
            'opt_cost': int(opt_cost),
            'gap_percent': gap,
            'time': exec_time
        })
        print(f"  [KẾT QUẢ] {algo_name:<20} | Cost: {int(cost):<8} | Gap: {gap:>6.2f}% | Time: {exec_time:>6.3f}s")

    # --- CHẠY 10 THUẬT TOÁN --- 
    
    # --- 1. Nearest Neighbor --- 
    print("  Đang chạy: Nearest Neighbor...")
    start_time = time.time()
    nn_tour, nn_cost = nearest_neighbor.solve(matrix, start_node=0)
    exec_time = time.time() - start_time
    record_result('Nearest Neighbor', nn_tour, nn_cost, exec_time)
    
    # --- 2. Nearest Insertion --- 
    print("  Đang chạy: Nearest Insertion...")
    start_time = time.time()
    ni_tour, ni_cost = nearest_insertion.solve(matrix, start_node=0)
    exec_time = time.time() - start_time
    record_result('Nearest Insertion', ni_tour, ni_cost, exec_time)

    # --- 3. Christofides --- 
    print("  Đang chạy: Christofides...")
    start_time = time.time()
    try:
        chris_tour, chris_cost = christofides.solve(matrix)
        exec_time = time.time() - start_time
        record_result('Christofides', chris_tour, chris_cost, exec_time)
    except ImportError as e:
        print(f"  BỎ QUA: Christofides thất bại (Thiếu thư viện? {e})")
    
    # --- 4. 2-Opt (từ kết quả NN) --- 
    print("  Đang chạy: 2-Opt (from NN)...")
    start_time = time.time()
    two_opt_tour, two_opt_cost = two_opt.solve(matrix, initial_tour=nn_tour)
    exec_time = time.time() - start_time
    record_result('2-Opt (from NN)', two_opt_tour, two_opt_cost, exec_time)

    # --- 5. Simulated Annealing (từ kết quả NN) --- 
    print("  Đang chạy: Simulated Annealing...")
    start_time = time.time()
    sa_tour, sa_cost = simulated_annealing.solve(matrix, initial_tour=nn_tour, **SA_PARAMS)
    exec_time = time.time() - start_time
    record_result('Simulated Annealing', sa_tour, sa_cost, exec_time)

    # --- 6. Genetic Algorithm --- 
    print("  Đang chạy: Genetic Algorithm...")
    start_time = time.time()
    ga_tour, ga_cost = genetic_algorithm.solve(matrix, **GA_PARAMS)
    exec_time = time.time() - start_time
    record_result('Genetic Algorithm', ga_tour, ga_cost, exec_time)

    # --- 7. Ant Colony Optimization --- 
    print("  Đang chạy: Ant Colony (ACO)...")
    # Điều chỉnh tham số ACO cho kích thước vấn đề
    aco_params_dynamic = ACO_PARAMS.copy()
    aco_params_dynamic['num_ants'] = dim # Đặt số kiến bằng số thành phố
    start_time = time.time()
    aco_tour, aco_cost = ant_colony.solve(matrix, **aco_params_dynamic)
    exec_time = time.time() - start_time
    record_result('Ant Colony (ACO)', aco_tour, aco_cost, exec_time)

    # --- 8. Tabu Search (từ kết quả NN) --- 
    print("  Đang chạy: Tabu Search...")
    # Điều chỉnh tham số TS cho kích thước vấn đề
    ts_params_dynamic = TS_PARAMS.copy()
    ts_params_dynamic['tabu_tenure'] = int(dim * 0.5) # Nhiệm kỳ cấm ~ N/2
    start_time = time.time()
    ts_tour, ts_cost = tabu_search.solve(matrix, initial_tour=nn_tour, **ts_params_dynamic)
    exec_time = time.time() - start_time
    record_result('Tabu Search', ts_tour, ts_cost, exec_time)

    # --- THUẬT TOÁN EXACT (CHẠY CÓ ĐIỀU KIỆN) --- 
    
    # --- 9. Brute Force --- 
    if dim <= 10: # Giới hạn N=10
        print("  Đang chạy: Brute Force (N<=10)...")
        start_time = time.time()
        bf_tour, bf_cost = brute_force.solve(matrix)
        exec_time = time.time() - start_time
        record_result('Brute Force', bf_tour, bf_cost, exec_time)
    else:
        print(f"  BỎ QUA: Brute Force (N={dim} > 10)")

    # --- 10. Held-Karp --- 
    if dim <= 21: # Giới hạn N=21
        print("  Đang chạy: Held-Karp (N<=21)...")
        start_time = time.time()
        hk_tour, hk_cost = held_karp.solve(matrix)
        exec_time = time.time() - start_time
        record_result('Held-Karp', hk_tour, hk_cost, exec_time)
    else:
        print(f"  BỎ QUA: Held-Karp (N={dim} > 21)")
        
    print(f"--- Hoàn thành {problem_name.upper()} ---\n")

print("\n--- TẤT CẢ THÍ NGHIỆM HOÀN TẤT ---\n")

## 4. Xử lý, Hiển thị và Lưu Kết quả

Bây giờ chúng ta chuyển danh sách kết quả (list of dicts) thành một DataFrame `pandas` đẹp đẽ, tính toán `gap %` và lưu vào file `results.csv`.

In [ ]:
# Chuyển đổi danh sách kết quả thành DataFrame
df_results = pd.DataFrame(all_results)

# Đảm bảo các cột có thứ tự hợp lý
columns_order = [
    'problem', 'dimension', 'algorithm', 
    'cost', 'opt_cost', 'gap_percent', 'time'
]
df_results = df_results[columns_order]

# Hiển thị toàn bộ DataFrame kết quả
print("--- BẢNG KẾT QUẢ TỔNG HỢP ---")
print(df_results.to_string())

# Lưu vào file CSV
try:
    df_results.to_csv(RESULTS_FILE, index=False)
    print(f"\nĐã lưu kết quả thành công vào file: {RESULTS_FILE}")
except Exception as e:
    print(f"\nLỖI: Không thể lưu file CSV. {e}")

## 5. Phân tích Nhanh

Hãy nhóm theo thuật toán và tính toán trung bình `gap_percent` và `time`.

In [ ]:
if not df_results.empty:
    print("\n--- PHÂN TÍCH NHANH (TRUNG BÌNH) ---")
    # Tính trung bình Gap và Thời gian cho mỗi thuật toán
    # Chúng ta chỉ tính trung bình cho các vấn đề Heuristic (N > 21)
    heuristic_results = df_results[df_results['dimension'] > 21]

    if not heuristic_results.empty:
        analysis = heuristic_results.groupby('algorithm')[['gap_percent', 'time']].mean()
        analysis = analysis.sort_values(by='gap_percent')
        print(analysis)
    else:
        print("Không có kết quả từ các vấn đề lớn (N>21) để phân tích.")
else:
    print("Không có kết quả nào được tạo ra.")